In [ ]:
import os
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

from glob import glob
from PIL import Image
from torch.utils.data import DataLoader, Dataset
import Augmentor

In [ ]:
data_augmentation_enabled = True

In [ ]:
def augment_images(input_dir, output_dir, num_augmentations=int(1e2)):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    for cls in range(1, 3):
        in_dir = input_dir + '/' + str(cls)
        print('Augmenting images in ' + in_dir)
        out_dir = output_dir + '/' + str(cls)
        print('Augmenting into ' + out_dir)
        if not os.path.exists(out_dir):
            os.makedirs(out_dir)
        p = Augmentor.Pipeline(source_directory=in_dir, output_directory="out")
        p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
        p.zoom_random(probability=0.5, percentage_area=0.8)
        p.flip_left_right(probability=0.3)
        p.random_distortion(probability=0.5, grid_width=4, grid_height=4, magnitude=1)
        p.skew_tilt(probability=0.5, magnitude=0.1)
        p.random_erasing(probability=0.2, rectangle_area=0.1)
        p.sample(num_augmentations)
        #out is relative for augmentator :/ move
        os.rename(os.path.join(input_dir,str(cls),'out'), os.path.join(output_dir,str(cls)))

# note that function will fail if augmentations were already present
if data_augmentation_enabled:
    augment_images('dev', 'dev/da')
    augment_images('train', 'train/da')

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, images, labels) -> None:
        self.images = images
        self.labels = labels

    def __getitem__(self, idx):
        label = self.labels[idx]
        image = self.images[idx]
        return image, label

    def __len__(self):
        return len(self.labels)

CLASSES = 2

def png_load(dir_name):
    """
    Loads all *.png images
    """
    features = {}
    for f in glob(dir_name + '/*.png'):
        features[f] = np.array(Image.open(f), dtype=np.float64)
    return features


In [ ]:


train_x = np.empty((0,80,80,3))
train_y = np.empty((0),dtype=int)

test_x = np.empty((0,80,80,3))
test_y = np.empty((0),dtype=int)

for i in range(1,CLASSES+1):
    train_i = np.array(list(png_load(os.path.join("train/da",str(i))).values()))
    label_i = np.full(len(train_i),i-1)
    train_x = np.concatenate((train_x, train_i), axis=0)
    train_y = np.concatenate((train_y,label_i), axis=0)

    test_i = np.array(list(png_load(os.path.join("dev/da",str(i))).values()))
    label_i = np.full(len(test_i),i-1)
    test_x = np.concatenate((test_x, test_i), axis=0)
    test_y = np.concatenate((test_y,label_i), axis=0)

print("Images were successfully loaded")

# convert 80,80,3 to 3,80,80
train_x = np.array(train_x)
train_x = np.transpose(train_x, (0, 3, 1, 2))

test_x = np.array(test_x)
test_x = np.transpose(test_x, (0, 3, 1, 2))

class SmallCNNMultiClass(nn.Module):
    def __init__(self, num_classes=2):
        super(SmallCNNMultiClass, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 17 * 17, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, num_classes)

    def forward(self, x):
        x = self.pool(nn.functional.relu(self.conv1(x)))
        x = self.pool(nn.functional.relu(self.conv2(x)))
        x = x.view(-1, 16 * 17 * 17)  # adjust shape to match fc1 input size
        x = nn.functional.relu(self.fc1(x))
        x = nn.functional.relu(self.fc2(x))
        x = self.fc3(x)
        return x
     #TODO pocet trid a softmax odstran

# Convert NumPy arrays to PyTorch tensors
train_tensors = torch.Tensor(train_x)
test_tensors = torch.Tensor(test_x)


# Create new TensorDataset instances with the modified labels
train_dataset = CustomDataset(train_tensors, train_y)
test_dataset = CustomDataset(test_tensors, test_y)
print("Dataset was successfully created")
# import random
# x, y = random.choice(train_dataset)
# print(y)

# import matplotlib.pyplot as plt

# plt.imshow(x[0,:,:])
# plt.savefig('idk.png')

batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
dev_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

model = SmallCNNMultiClass()
criterion = F.cross_entropy
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
num_epochs = 50
for epoch in range(num_epochs):
    model.train()
    train_loss = 0
    for inputs, labels in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss += loss.item()

    if not epoch % 10:
        # Evaluation on the dev set
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in dev_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()

        accuracy = correct / total
        print(f'Epoch: {epoch}/{num_epochs}, Loss: {train_loss:.4f}, Accuracy: {accuracy:.4f}')